In [7]:
from langchain_mistralai.chat_models import ChatMistralAI
from openai import AsyncAzureOpenAI
from openai import OpenAI, AsyncOpenAI
import instructor

import asyncio
import nest_asyncio

nest_asyncio.apply()

client = OpenAI(
    base_url="https://Mistral-large-vigie-ai-serverless.francecentral.inference.ai.azure.com/v1", api_key="qe1IoBAEweX7bZ0vpneDFICf32UyaNTm"
)

GENERATION_ENGINE = "Mistral-large-vigie-ai"

In [5]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Who is the most renowned French painter? Provide a short answer.",
        }
    ],
    model="azureai",
)

In [6]:
response

ChatCompletion(id='3193971b-4550-4086-85ca-45266ec7fc6a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=' The most renowned French painter is widely considered to be Claude Monet, one of the founders of the Impressionist movement.', role='assistant', function_call=None, tool_calls=None))], created=252406, model='mistral-large', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=18, total_tokens=46))

In [8]:
async def get_async_analysis(prompt, response_model):
    response: response_model = await client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Tu est un assistant spélialisé dans l'analyse de commentaires, et qui ne renvoit que des fichiers JSON.",
            },
            {"role": "user", "content": str(prompt)},
        ],
        response_format={"type": "json_object"},
        model=GENERATION_ENGINE,
        temperature=TEMPERATURE,
        # max_retries=MAX_RETRIES,
        response_model=response_model,
    )
    return response  # .choices[0].message.content

def apply_async_analysis(prompts, response_models):
    if not response_models.isinstance(list):
    #if type(response_models) is not list:
        response_models = [response_models for _ in prompts]
    loop = asyncio.get_event_loop()
    tasks = [
        loop.create_task(get_async_analysis(prompt, response_model))
        for (prompt, response_model) in zip(prompts, response_models)
    ]
    res = loop.run_until_complete(asyncio.gather(*tasks))
    return res